In [37]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "input_data": {
    "columns": [
      "date",
      "quarter",
      "department",
      "day",
      "team",
      "targeted_productivity",
      "smv",
      "wip",
      "over_time",
      "incentive",
      "idle_time",
      "idle_men",
      "no_of_style_change",
      "no_of_workers"
    ],
    "index": [0,1],
    "data": [
      ["1/3/2015", "Quarter1", "finishing", "Saturday", 9, 0.8, 4.15, 0, 960, 0, 0, 0, 0, 8], 
      ["1/1/2015","Quarter1","finishing","Thursday", 7, 0.8, 2.9, 0, 960, 0, 0, 0, 0, 8]
    ]
  }

}

body = str.encode(json.dumps(data))

url = 'https://regression-sbqey.eastus2.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'HERE SHOULD BE THE PRIMARY KEY FROM AZURE'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'automl15d39d6b340-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[0.6670469952221965, 0.6005908728283333]'
